<a href="https://www.kaggle.com/code/bromoshiki/monunet-mpmc?scriptVersionId=156905074" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import packages

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import Resizing, Rescaling, RandomFlip, RandomRotation
from tensorflow.keras import Sequential
from tensorflow.keras.applications import Xception,resnet
from tensorflow.keras.applications.inception_v3 import InceptionV3 
from tensorflow.keras.applications.vgg19 import VGG19,preprocess_input
from tensorflow.keras.applications import efficientnet
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input,DepthwiseConv2D, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape,ReLU,AvgPool2D,Layer)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import itertools

In [ ]:
%matplotlib inline

matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [ ]:
IMAGE_SIZE = 256

# Split the dataset into training and testing

In [ ]:
train_dir = "/kaggle/input/monument-dataset/dataset_monunet"

CONFIGURATION = {
    "BATCH_SIZE": 8,
    "IMAGE_SIZE": 256,
    "LEARNING_RATE": 1e-3,
    "EPOCHS": 50,
    "DROPOUT_RATE": 0.0,
    "REGULARIZATION_RATE": 0.0,
    "N_FILTERS": 6,
    "KERNEL_SIZE": 3,
    "N_STRIDES": 1,
    "POOL_SIZE": 2,
    "N_DENSE_1": 1024,
    "N_DENSE_2": 128,
    "NUM_CLASSES": 13,
    "PATCH_SIZE": 16,
    "PROJ_DIM": 768,
    "class_names" : ['Belur Math,Kolkata,India','Black Hole Monument,Kolkata,India','Dakshineswar Kali Temple,Kolkata,India','Fort William,Kolkata,India','Howrah Bridge,Kolkata,India','James Princep Memorial,Kolkata,India','Marble Palace,Kolkata,India','Metcalfe Hall,Kolkata,India','Raj Bhavan,Kolkata,India','Shaheed Minar,Kolkata,India','St. Pauls Cathedral,Kolkata,India','Victoria Memorial,Kolkata,India','Writer’s Building,Kolkata,India']
}

In [ ]:
train_ds=tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    class_names=CONFIGURATION["class_names"], 
    color_mode='rgb',
    batch_size=CONFIGURATION["BATCH_SIZE"],
    image_size=(CONFIGURATION["IMAGE_SIZE"],CONFIGURATION["IMAGE_SIZE"]),
    shuffle=True,
    seed=99,
)

In [ ]:
len(train_ds)

In [ ]:
def data_split(dataset,TRAIN_RATIO,VAL_RATIO,TEST_RATIO):
    size=len(dataset)
    train_dataset=dataset.take(int(TRAIN_RATIO*size))
    val_dataset=dataset.skip(int(TRAIN_RATIO*size))
    test_dataset=val_dataset.skip(int(TEST_RATIO*size))
    test_dataset=test_dataset.take(int(TEST_RATIO*size))
    val_dataset=val_dataset.take(int(VAL_RATIO*size))
    return train_dataset,val_dataset,test_dataset
train_ds,val_ds,test_ds=data_split(train_ds,0.8,0.1,0.1)

# Resizing, Rescaling and data augmentation

In [ ]:
train_ds = train_ds.cache().shuffle(2000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(500).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
from tensorflow.keras import models, layers
resize_and_rescale = Sequential([
    Resizing(IMAGE_SIZE, IMAGE_SIZE),
    Rescaling(1.0/255) 
])

In [ ]:
class Bright(Layer):
    def __init__(self):
        super().__init__()
        
    def call(self, image):
        return tf.image.adjust_brightness(image, 0.1)

In [ ]:
data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal_and_vertical"), #horizontal and vertical flip
    RandomRotation(0.2), #Rotates
    # Bright()
])

# Model creation

In [ ]:
backbone = InceptionV3(
    include_top = False,
    weights='imagenet',
    input_shape=(CONFIGURATION["IMAGE_SIZE"], CONFIGURATION["IMAGE_SIZE"], 3),
    )
     
backbone.trainable = False
     
model = tf.keras.Sequential([
    Input(shape = (256, 256,3)),
    resize_and_rescale,
    data_augmentation,
    backbone,
    
#     Conv2D(32,(3,3),activation="relu",padding="same"),
#     MaxPooling2D(3,3),
    
#     Conv2D(64,(3,3),activation="relu",padding="same"),
#     Conv2D(64,(3,3),activation="relu",padding="same"),
    
#     MaxPooling2D(2,2),
    
    GlobalAveragePooling2D(),
    
    Dense(1024, activation = "relu"),
    
    BatchNormalization(),
    
    Dense(128, activation = "relu"),
    Dense(13, activation = "softmax"),
    ])

model.summary()

# Training Analysis through Metrics

In [ ]:
class Precision(tf.keras.metrics.Metric):
    def __init__(self, name='precision', **kwargs):
        super(Precision, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(tf.greater(y_pred, 0.5), tf.float32)
        tp = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 1)), tf.float32))
        fp = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 0), tf.equal(y_pred, 1)), tf.float32))
        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)

    def result(self):
        return self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())

In [ ]:
class Recall(tf.keras.metrics.Metric):
    def __init__(self, name='recall', **kwargs):
        super(Recall, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(tf.greater(y_pred, 0.5), tf.float32)
        tp = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 1)), tf.float32))
        fn = tf.reduce_sum(tf.cast(tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 0)), tf.float32))
        self.true_positives.assign_add(tp)
        self.false_negatives.assign_add(fn)

    def result(self):
        return self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())

In [ ]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.precision = Precision()
        self.recall = Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision.update_state(y_true, y_pred)
        self.recall.update_state(y_true, y_pred)

    def result(self):
        precision = self.precision.result()
        recall = self.recall.result()
        return 2 * ((precision * recall) / (precision + recall + tf.keras.backend.epsilon()))

# Model Compilation

In [ ]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False), 
    metrics=["accuracy",Precision(), Recall(), F1Score()]  #metrics used to track training
)

# Model Training

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
filepath = './best_weights_EFNB3_HUMAN_ACTION.hdf5'

earlystopping = EarlyStopping(monitor = 'val_accuracy', 
                              mode = 'max' , 
                              patience = 15,
                              verbose = 1)

checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_accuracy', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)

learning_rate = ReduceLROnPlateau(monitor = 'val_accuracy',
                                  mode = 'max',
                                  patience = 5,
                                  factor = 0.3,
                                  min_delta = 0.00001)


callback_list = [earlystopping, checkpoint, learning_rate]

In [ ]:
EPOCHS = 12
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=8,
    verbose=1,  
    validation_data=val_ds,
    callbacks = callback_list
)

In [ ]:
def display_history(history_):
    fig, ax = plt.subplots(1,2, figsize=(12, 3))
    ax[0].plot(history_.history['loss'], color='b', label="training_loss")
    ax[0].plot(history_.history['val_loss'], color='r', label="validation_loss",axes =ax[0])
    ax[0].set_xlabel("Epoch")
    ax[0].set_ylabel("Loss")
    legend = ax[0].legend(loc='best', shadow=True)
    ax[1].plot(history_.history['accuracy'], color='b', label="training_accuracy")
    ax[1].plot(history_.history['val_accuracy'], color='r',label="validation_accuracy")
    ax[1].set_xlabel("Epoch")
    ax[1].set_ylabel("Accuracy")
    legend = ax[1].legend(loc='best', shadow=True)


# Plotting Metrics

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train_loss','val_loss'])
plt.show()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('epoch')
plt.legend(['train_acc','val_acc'])
plt.show()

In [ ]:
plt.plot(history.history['precision'])
plt.plot(history.history['val_precision'])
plt.title('Model Precision')
plt.ylabel('Precision')
plt.xlabel('epoch')
plt.legend(['train_prec','val_prec'])
plt.show()

In [ ]:
plt.plot(history.history['recall'])
plt.plot(history.history['val_recall'])
plt.title('Model Recall')
plt.ylabel('Recall')
plt.xlabel('epoch')
plt.legend(['train_recall','val_recall'])
plt.show()

In [ ]:
plt.plot(history.history['f1_score'])
plt.plot(history.history['val_f1_score'])
plt.title('Model F1_Score')
plt.ylabel('F1_Score')
plt.xlabel('epoch')
plt.legend(['train_f1_score','val_f1_score'])
plt.show()

# Model Evaluation and Prediction

In [ ]:
model.evaluate(test_ds)

In [ ]:
# plt.figure(figsize=(24,24))
# for images, labels in test_ds.take(1):
#     for i in range(min(16, len(images))):
#         ax = plt.subplot(4,4,i+1)
#         plt.imshow(images[i]/255.)
#         plt.title("True Label : "+ CONFIGURATION["class_names"][tf.argmax(labels[i], axis=-1).numpy()] + "\n" + "Predicted Label : " + CONFIGURATION["class_names"][int(tf.argmax(model(tf.expand_dims(images[i], axis=0)), axis=-1).numpy()[0])])

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

image_path = '/kaggle/input/niggawhat/dos.png'
img = load_img(image_path, target_size=(256, 256))
img = img_to_array(img)
img = np.expand_dims(img, axis=0)

prediction = model.predict(img)

In [ ]:
class_labels = ['Belur Math,Kolkata,India','Black Hole Monument,Kolkata,India','Dakshineswar Kali Temple,Kolkata,India','Fort William,Kolkata,India','Howrah Bridge,Kolkata,India','James Princep Memorial,Kolkata,India','Marble Palace,Kolkata,India','Metcalfe Hall,Kolkata,India','Raj Bhavan,Kolkata,India','Shaheed Minar,Kolkata,India','St. Pauls Cathedral,Kolkata,India','Victoria Memorial,Kolkata,India','Writer’s Building,Kolkata,India']  # Replace with your actual class labels

predicted_class_index = np.argmax(prediction)
predicted_label = class_labels[predicted_class_index]

print("Predicted Label:", predicted_label)

# Saving model

In [ ]:
# model.save('MonuNetv3.h5')

In [ ]:
from tensorflow.keras.models import load_model

# Save the model
model.save('efficientnetB3_kolkata_model.h5')

In [ ]:
from PIL import Image
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf


# Load your Keras model
model = load_model('/kaggle/working/best_weights_EFNB3_HUMAN_ACTION.hdf5')


# Define your class labels
class_labels = ['Belur Math,Kolkata,India','Black Hole Monument,Kolkata,India','Dakshineswar Kali Temple,Kolkata,India','Fort Williams,Kolkata,India','Howrah Bridge,Kolkata,India','James Princep Memorial,Kolkata,India','Marble Palace,Kolkata,India','Metcalfe Hall,Kolkata,India','Raj Bhavan,Kolkata,India','Shaheed Minar,Kolkata,India','St. Pauls Cathedral,Kolkata,India','Victoria Memorial,Kolkata,India','Writers Building,Kolkata,India']


image_path = '/kaggle/input/niggawhat/dos.png'
image = load_img(image_path, target_size=(300, 300))
# Preprocess the image (e.g., resize to match your model's input size)
image = image.resize((256, 256))  # Adjust dimensions as needed
x = img_to_array(image)
ximg = np.expand_dims(img, axis=0)

predictions = model.predict(x)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)
print(f"Predicted class index: {predicted_class_index}")
predicted_label = class_labels[predicted_class_index]
print(f"Predicted label: {predicted_label}")        
